In [1]:
import os
import mlflow
import numpy as np
import dask.dataframe as dd

In [2]:
import sys
sys.path.append("..")
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs


In [3]:
from config import settings

In [4]:

where_is = WhereIs(False)

In [5]:
all_paths = where_is.get_paths_list

In [6]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0])


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
customers_df = load_customers(all_paths[1])

In [8]:
test_file, lnr = load_test_file(all_paths[6])

In [9]:
clean_up = CleanUp(all_paths)



In [10]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df, 'test_file': test_file}

In [11]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_dict = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[5], info_df=all_paths[9],attr_df=all_paths[8])

/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].replace(value, np.nan)


In [12]:
from models.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool

In [13]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())


I see 1 GPU devices


In [14]:
df_to_model = dd.concat(
    [dd.from_pandas(dfs_dict['customers_df'].assign(is_customer=1), npartitions=10),
     dd.from_pandas(dfs_dict['azdias'].assign(is_customer=0), npartitions=10)]
)



In [15]:
cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

In [16]:
cat_features

Index(['CAMEO_DEUG_2015', 'CAMEO_DEUINTL_2015', 'CAMEO_DEU_2015',
       'OST_WEST_KZ'],
      dtype='object')

In [ ]:
data_types_bool = {key: {isinstance(item, (int,str)) for item in df_to_model[key].value_counts().index.tolist()} for key in cat_features}


In [ ]:
data_types = {key: {item for item in df_to_model[key].value_counts().index.tolist()} for key in cat_features}

In [ ]:
## Apagar depois, experimento de tudo virar int
df_to_transform = dfs_dict['customers_df'].apply(pd.to_numeric, errors='coerce')

In [17]:
nans = df_to_model.CAMEO_DEUINTL_2015.isna().sum()

In [2]:
df_categorized = df_to_model.categorize(columns=cat_features)

NameError: name 'df_to_model' is not defined

In [26]:
df_to_model.CAMEO_DEUINTL_2015.cat.known

AttributeError: Can only use .cat accessor with a 'category' dtype

In [18]:
for cat in cat_features:
    df_to_model[cat] = (
                    df_to_model[cat].cat.add_categories('UNKNOWN').fillna('UNKNOWN')
                )

AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:
#b = {isinstance(item, (int,str)) for item in df_to_model[cat_features[0]].value_counts().index.tolist()}

In [ ]:
features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')


In [ ]:
X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

In [ ]:
#Investigating error
X_train[cat_features].isna().sum().sum()

In [ ]:
experiment_id = mlflow.create_experiment("CatBoostClassifier")
experiment = mlflow.get_experiment(experiment_id)


In [ ]:
cat_boost_params = {'num_trees':40,
                    'task_type':'GPU',
                    'max_depth': 3,
                    'in_child_samples': 30,
                    'one_hot_max_size': 5,
                    'class_weights':class_weights,
                    'cat_features':cat_features,
                    'random_state':settings.RANDOM_STATE,
                    'eval_metric':'AUC',
                    'od_type':'Iter',
                    'od_wait':40}

In [21]:
with mlflow.start_run(experiment):
    mlflow.log_params(cat_boost_params)
    mlflow.set_tags({'sample_ratio': 1, 'dataset_for_train':'population+customers','columns_used':X_train.columns, 'columns_type': ['object', 'category']})
    catboost = CatBoostClassifier(cat_boost_params)
    catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)
    logs_metrics = show_metrics_baseline(catboost, features=features, labels=labels)
    mlflow.log_metrics(logs_metrics)

Accuracy Train: 0.7634742876534681
AUC Train: 0.8651072729262171
Accuracy Test: 0.7618845620881762
AUC Test: 0.8653336607429379
Accuracy Valid: 0.7618562823449563
AUC Valid: 0.8649432273989134
